In [1]:
%run utils.ipynb
%run data_encoder.ipynb

In [2]:
import midi
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.externals import joblib

In [3]:
%matplotlib notebook

In [4]:
classifier = joblib.load('../models/es8-singlenotes.model')

In [20]:
filename = '../data/ES8-SingleNotes/notes.wav'#'../test/single-notes-001.wav'
sampling_rate, wav_data = wavfile.read(filename)
wav_data = wav_data[:, 0]
samples_count = len(wav_data)
num_ticks = 7000

In [70]:
notes = []
prev_max = -1
prev_tick = -0
for idx in range(0, samples_count - num_ticks, 100): # intervals of ~2.25 ms (441 times per second)
    wav = wav_data[max(0, idx - 1000 + 1) : idx + 500]
    new_max = np.max(np.abs(wav))
    
    # Not a note
    if new_max <= 200:
        continue
    
    # Same note as before
    if new_max < prev_max:
        prev_max = new_max
        continue
    
    # Not significant enough to be a new note
    if new_max - prev_max < 400:
        continue
    
    prev_mx = new_maxa
    
    if prev_tick > 0 and idx - prev_tick < 1000:
        continue
    
    prev_tick = idx
    wav = wav_data[idx : idx + num_ticks]
    data = encode_data(wav, sampling_rate)
    notes.append((idx, classifier.predict([data])[0]))
    
    #print("Note at {}: {} vs {}".format(idx, prev_max, new_max))
    

In [71]:
for tick, note in notes:
    print('Tick {}: {}'.format(tick, es8_piano_note_decoder(note)))

Tick 1500: ('C', 2)
Tick 89600: ('C', 2)
Tick 177600: ('C', 2)
Tick 265800: ('C', 2)
Tick 267400: ('C', 2)
Tick 274400: ('C', 2)
Tick 355500: ('C#', 2)
Tick 442200: ('C#', 2)
Tick 530400: ('C#', 2)
Tick 618500: ('C#', 2)
Tick 620100: ('C#', 2)
Tick 624300: ('C#', 2)
Tick 635000: ('C#', 2)
Tick 795000: ('D', 2)
Tick 796400: ('D', 2)
Tick 883200: ('D', 2)
Tick 971300: ('D', 2)
Tick 972800: ('D', 2)
Tick 1147900: ('D#', 2)
Tick 1151300: ('D#', 2)
Tick 1236000: ('D#', 2)
Tick 1239400: ('D#', 2)
Tick 1324000: ('D#', 2)
Tick 1326000: ('D#', 2)
Tick 1500700: ('E', 2)
Tick 1588700: ('E', 2)
Tick 1676900: ('E', 2)
Tick 1684400: ('E', 2)
Tick 1853400: ('F', 2)
Tick 1941500: ('F', 2)
Tick 1943900: ('F', 2)
Tick 2029600: ('F', 2)
Tick 2031600: ('F', 2)
Tick 2045800: ('F', 2)
Tick 2058500: ('F', 2)
Tick 2206200: ('F#', 2)
Tick 2294400: ('F#', 2)
Tick 2382500: ('F#', 2)
Tick 2409200: ('F#', 2)
Tick 2559000: ('G', 2)
Tick 2647100: ('G', 2)
Tick 2735300: ('G', 2)
Tick 2737600: ('G', 2)
Tick 2742100: (

In [72]:
pattern = midi.Pattern(resolution=441)
track = midi.Track()
pattern.append(track)
track.append(midi.SetTempoEvent(tick=0, bpm=60))

In [73]:
prev_time = 0
for note in notes[0 : 100]:
    track.append(midi.NoteOnEvent(tick=(note[0]-prev_time)//100, pitch=note[1], velocity=60))
    prev_time = note[0]

In [74]:
track.append(midi.EndOfTrackEvent(tick=1))

# Save the pattern to disk
midi.write_midifile('../predictions/test-000.midi', pattern)